In [7]:
import json
import re
import html
import gevent
from time import time
import codecs
import jieba
jieba.enable_parallel(8)
class ReadFromNLine:
    reReplacePat = [(re.compile(r'(<br\s?/>)+'), r'\n'),
                    (re.compile(r'<[^>]*>'), r''),
                    (re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)'), r''),
                    (re.compile(r'(\s*[\r|\n]+\s*)+'), r'\n'),]
    def __init__(self, file, skip_nline = 0, get_nline = 1):
        self.skNline = skip_nline
        self.getNline = get_nline
        self.file = file
        self.output = [None] * get_nline
    def extract(self):
        with open(self.file, 'rb') as f:
            for i in range(self.skNline): next(f)
            l = 0
            for line in f:
                try:
                    js = json.loads(line.decode("utf-8"))
                    content = self.clean(js['content'])
                    self.output[l] = {'category': js['category'], 'content': content, 'success': True}
                except:
                    print('preprocess failed at line: ' + str(l + self.skNline))
                    self.output[l] = {'category': '', 'content': '', 'success': False}
                # print(js['category'])
                # print(js['url'])
                # print(repr(content))
                l += 1
                if l >= self.getNline:
                    break
        return self.output
    def clean(self, rawContent):
        content = html.unescape(rawContent)
        for pat, rep in ReadFromNLine.reReplacePat:
            content = re.sub(pat, rep, content).strip()
        return content
        


        


In [386]:
file = 'data/2016-hackathon-qa.jsonl.raw'
t = time()
preproc = ReadFromNLine(file, skip_nline=0, get_nline=168613)
o = preproc.extract()
print(time()-t)

with codecs.open('data/clean-qa.jsonl', 'wb', encoding='utf-8') as f:
    for d in o:
        if d['success']:
            f.write(json.dumps({'category': d['category'], 'content': d['content']}) + '\n')

189.8424587249756


In [412]:
# import jieba
# t = time()
# with open('data/clean-qa-jq-cn.jsonl', 'rb') as f:
#     with codecs.open('data/jb-cn.txt', 'wb', encoding='utf-8') as g:
#         for line in f:
#             js = json.loads(line.decode("utf-8"))
#             txt = ' '.join(jieba.cut(js['content']))
#             g.write(txt + '\n')
# print(time()-t)


2402.3665919303894


In [4]:
# a = '!!!\r\n\r\n\xa0\r\n髮朵Phyto櫃點:\xa0\r\n髮朵PhytoFB官方網站:\r\n別忘了'
# print(repr(a))
# b = re.sub(r'(\s*[\r|\n]+\s*)+', '\n', a)
# print(repr(b))
# jieba.load_userdict('data/zhwiki-cn-clean')
jieba.load_userdict('data/dict-txt-big')

In [8]:
t = time()
with open('data/content-cn-lower.txt', 'rb') as f:
    with codecs.open('data/jb-cn.txt', 'wb', encoding='utf-8') as g:
        for line in f:
            txt = ' '.join(jieba.cut(line.decode("utf-8")))
            g.write(txt)
print(time()-t)

10146.80155801773
